In [1]:
%reload_ext autoreload
%autoreload 2

In [15]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import pandas as pd
import numpy as np
import re 
import joblib

curruser = os.environ.get('USER')

if curruser in os.listdir("/opt/workspace/"):
    sys.path.insert(0, '/opt/workspace/{user}/system/support_library/'.format(user=curruser))
    sys.path.insert(0, '/opt/workspace/{user}/libs/'.format(user=curruser))
    sys.path.insert(0, '/opt/workspace/{user}/system/labdata/lib/'.format(user=curruser))
else:
    sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser))
    sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
    sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

# # sys.path.insert(0, './../src')
# sys.path.insert(0, '/home/{user}/support_library/'.format(user=curruser))
# sys.path.insert(0, '/home/{user}/python35-libs/lib/python3.5/site-packages/'.format(user=curruser))
# sys.path.insert(0, '/home/{user}/notebooks/labdata/lib/'.format(user=curruser))


from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()
from corpora_process.utils import extend

from spark_connector import SparkConnector
from sparkdb_loader import spark
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame

import loader as load

import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [18]:
recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

### Connect

In [19]:
%%time
sp = spark(**SPARK_CONFIG)
print(sp.sc.version)
hive = sp.sql

2.4.0.cloudera2
CPU times: user 47.2 ms, sys: 43.1 ms, total: 90.3 ms
Wall time: 28.4 s


In [16]:
sp.exit()

type n to save the data, any other key will clear the screen


### Export2Iskra

In [20]:
schema     = 'sbx_team_digitcamp'
table_name = 'RECSYS_STORY_RATINGS_CUT'

fin_df = hive.sql('''select * from {}.{} '''.format(schema,table_name))

In [21]:
print(fin_df.count())

fin_df.show()

3152721


,INN,PRODUCT_ID,PRODUCT_RATING,TRUE_RATING,PRODUCT_RATING_NORM,agr_flg
0,0101005622,merchant-acquiring,0.006173,0.0,0.273802,0
1,010103649080,guarantee,0.001092,0.0,0.075597,0
2,010103807160,rabota_ru,0.005545,0.0,0.249282,0
3,010401650891,sberrating,0.006176,0.0,0.273891,0
4,010401823664,two_gis,0.000549,0.0,0.054387,0
5,010405139705,merchant-acquiring,0.003657,0.0,0.175650,0
6,010407937280,lyurist,0.004365,0.0,0.203261,0
7,0105007010,credit_card,0.007153,0.0,0.312037,0
8,0105022241,credit_card,0.007004,0.0,0.306211,0
9,010503859437,evotor,0.002074,0.0,0.113877,0


In [22]:
cols = fin_df.columns

fin_df = fin_df.select([col.upper() for col in fin_df.columns])

typesmap={}
for column_name, column in fin_df.dtypes:
    if column == 'string':
        if 'INN' in column_name.upper() or 'KPP' in column_name.upper():
            typesmap[column_name] = 'VARCHAR(20)'
        elif 'commonSegmentoUID'.upper() in column_name.upper():
            typesmap[column_name] = 'VARCHAR(4000)'
        else:
            typesmap[column_name] = 'VARCHAR(900)'
    elif column == 'int':
        typesmap[column_name] = 'INTEGER'
    elif column == 'bigint':
        typesmap[column_name] = 'INTEGER'
    elif column == 'timestamp':
        typesmap[column_name] = 'TIMESTAMP'
    elif column == 'float' or column == 'double' or column == 'decimal(9,2)':
        typesmap[column_name] = 'FLOAT'        
    else:
        None  

cols = ', '.join([col + ' ' + typesmap[col] for col in fin_df.columns])

In [23]:
cols

'INN VARCHAR(20), PRODUCT_ID VARCHAR(900), PRODUCT_RATING FLOAT, TRUE_RATING FLOAT, PRODUCT_RATING_NORM FLOAT, AGR_FLG INTEGER'

In [24]:
from connector import OracleDB

db = OracleDB('iskra4')
mode = 'overwrite'
fin_df \
    .write \
    .format('jdbc') \
    .mode(mode) \
    .option('url', 'jdbc:oracle:thin:@//'+db.dsn) \
    .option('user', db.user) \
    .option('password', db.password) \
    .option('dbtable', table_name) \
    .option('createTableColumnTypes', cols)\
    .option('driver', 'oracle.jdbc.driver.OracleDriver') \
    .save()

### Show result in oreacle db

In [25]:
db = OracleDB('iskra4')
res_table = sp.get_oracle(db, table_name)
res_table.show()

,INN,PRODUCT_ID,PRODUCT_RATING,TRUE_RATING,PRODUCT_RATING_NORM,AGR_FLG
0,010300802770,korpkarta,0.0014565883,0E-10,0.0898065930,0
1,0104012018,rabota_ru,0.0098146517,0E-10,0.4158520265,0
2,010401462626,korpkarta,0.0019089939,0E-10,0.1074547958,0
3,010401462626,rabota_ru,0.0035481825,0E-10,0.1713990299,0
4,010407371132,sberrating,0.0068087131,0E-10,0.2985913039,0
5,010501271880,lyurist,0.0080228951,0E-10,0.3459561632,0
6,0105013663,lyurist,0.0027206216,0E-10,0.1391161386,0
7,010502025333,two_gis,0.0012990658,0E-10,0.0836616895,0
8,0105044189,credit_card,0.0063088490,0E-10,0.2790917617,0
9,0105054282,zarplata,0.0133866351,0E-10,0.5551939803,1


In [26]:
table_name

'RECSYS_STORY_RATINGS_CUT'